In [1]:
import torch
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.48216, 0.44653),
                          (0.24703, 0.24349, 0.26159))])

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

Files already downloaded and verified


In [4]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

Files already downloaded and verified


In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=4)

In [6]:
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=4)

#### Inspecting the dataloader

In [7]:
print(trainloader.dataset.data.shape, testloader.dataset.data.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)


In [8]:
print(testloader.batch_size)

32


In [9]:
print(trainloader.sampler)

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=3e-4)

#loop over the dataset multiple times
for epoch in range(10):
    correct, total = 0,0
    predictions = []
    net.eval()
    for i, data in enumerate(trainloader, 0):
        #Get the inputs
        inputs, labels = data
        inputs = inputs.view(-1, 32 * 32 * 3)

        #zero the parameter gradients
        optimizer.zero_grad()

        #Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        predicted = torch.max(outputs.data, 1)
        predictions.append(outputs)
        total += labels.size(0)
        

print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))